In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mlp
from functools import partial
import seaborn as sns
import requests
from data_loading import *
import plotly.graph_objects as go
import os
import os.path as osp
import re

In [ ]:
# use this to keep the cells wider, very nice on wide screens, set the percentage as you like it
from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:80% !important; }</style>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
conf = {
    'font.size': 14.0,
    'axes.grid': True,
    'axes.axisbelow': True,
    'axes.edgecolor': 'black',
#    'axes.facecolor': '#E5E5EF',
#    'axes.facecolor': '#E0E0EF',
    'axes.labelcolor': 'black',
    'axes.titlesize': 15.6,
    'axes.labelsize': 'large',    
    'figure.figsize': (12, 7),
    'figure.titlesize': 'x-large',
    'grid.linewidth': 1.3,
    'xtick.labelsize': 'large',
    'ytick.labelsize': 'large',
    'xtick.color': 'black',
    'ytick.color': 'black',
    'legend.fontsize': 'large',
    'figure.figsize': (12., 8.)
}

plt.rcParams.update(conf)

### loading cleaned data

In [ ]:
df_form, columns_series, program_columns_ratings, program_columns, form_ratings, form_attends, open_answers = load_form_data()
df_app = load_app_data()
df_timetable = load_api_info(df_form.columns)
app_ratings = prepare_app_ratings(df_app)
program_ratings, program_attends, program_columns = merge_ratings(form_ratings, app_ratings)
program_fun_score, program_info_score = calc_scores(program_ratings, program_columns)

### app feedback analysis

In [ ]:
df_form.columns[df_form.columns.to_series().str.lower().str.contains('app')]

In [ ]:
app_columns = df_form.columns[df_form.columns.to_series().str.lower().str.contains('aplika')]
app_columns

In [ ]:
df_form[app_columns].head()

In [ ]:
def has_app_mention(ser):
    lower_str = ser.fillna('').astype(str).str.lower()
    return lower_str.str.contains('app') | lower_str.str.contains('aplika')

In [ ]:
app_mention_columns = [column for column in df_form.columns if has_app_mention(df_form[column]).any()]
app_mention_columns

In [ ]:
df_form[has_app_mention(df_form[app_mention_columns[0]])][app_mention_columns[0]]

In [ ]:
data_dir = osp.join(os.getcwd(), 'app_feedback')

In [ ]:
df_form['Používali jste Animefest aplikaci?'].value_counts().plot.pie()
plt.savefig(osp.join(data_dir, 'app_usage.png'))

In [ ]:
df_form[df_form['Co se vám na aplikaci líbilo? Co chcete do příště změnit nebo doplnit?'] != '']['Co se vám na aplikaci líbilo? Co chcete do příště změnit nebo doplnit?'].head()

In [ ]:
df_form[df_form['Narazili jste při používání aplikace na závažné problémy?'].notnull()]['Narazili jste při používání aplikace na závažné problémy?'].value_counts().plot.pie()
plt.savefig(osp.join(data_dir, 'app_bugs.png'))

In [ ]:
df = df_form[['Věk', 'Pohlaví', 'Kromě letošního ročníku jsem na Animefestu byl(a)', 
              'Používali jste Animefest aplikaci?', 'Narazili jste při používání aplikace na závažné problémy?']
            ].dropna(subset=['Kromě letošního ročníku jsem na Animefestu byl(a)', 'Používali jste Animefest aplikaci?'])
df['Kromě letošního ročníku jsem na Animefestu byl(a)'] = df['Kromě letošního ročníku jsem na Animefestu byl(a)'].replace({'6x': '6x+', '7x': '6x+', '8x': '6x+', '9x': '6x+', 
                                                                                                                           '10x': '6x+', '11x': '6x+', '12x': '6x+', '13x': '6x+', '14x': '6x+'})
# Create dimensions
age_dim = go.parcats.Dimension(
    values=df['Věk'],
    categoryorder='category ascending', label='Věk'
)

gender_dim = go.parcats.Dimension(
    values=df['Pohlaví'],
    categoryorder='category ascending', label='Pohlaví'
)

num_visits_dim = go.parcats.Dimension(
    values=df['Kromě letošního ročníku jsem na Animefestu byl(a)'],
    categoryorder='category ascending', label='Kolikrát předtím na AF'
)

usage_dim = go.parcats.Dimension(
    values=df['Používali jste Animefest aplikaci?'],
    categoryorder='category ascending', label='Používanost aplikace'
)

bugs_dim = go.parcats.Dimension(
    values=df['Narazili jste při používání aplikace na závažné problémy?'],
    categoryorder='category ascending', label='Problémy s aplikací'
)

# Create parcats trace
#color = df_form['Pohlaví'].replace({'muž': 0, 'žena': 0.3, 'jiné': 0.6, np.nan: 1})
color = df['Narazili jste při používání aplikace na závažné problémy?'].replace({'Vůbec': 0, 'Zřídka': 0.3, 'Často': 0.6, np.nan: 0.9})
#colorscale = [[0, 'steelblue'], [0.3, 'indianred'], [0.6, 'gray'], [1, 'dimgray']];

fig = go.Figure(data = [go.Parcats(dimensions=[age_dim, gender_dim, num_visits_dim, usage_dim, bugs_dim],
#        line={'color': color, 'colorscale': colorscale},
        line={'color': color},
        hoveron='color', hoverinfo='count+probability',
        labelfont={'size': 18, 'family': 'Times'},
        tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])
fig.show()

img_path = osp.join(data_dir, 'app_parallel_cords.png')
fig.write_image(img_path, width=1000, height=600)

# need to rotate it
par_cat = Image.open(img_path)
par_cat = par_cat.transpose(Image.ROTATE_90)
par_cat.save(img_path)

In [ ]:
from jinja2 import Environment, PackageLoader, select_autoescape, FileSystemLoader
from PIL import Image

def generate_pdf():
    pd.set_option('display.max_colwidth', -1)
    
    app_feedback_df = df_form[['Používali jste Animefest aplikaci?', 'Narazili jste při používání aplikace na závažné problémy?', 'Co se vám na aplikaci líbilo? Co chcete do příště změnit nebo doplnit?']]
    app_feedback_df = app_feedback_df[app_feedback_df['Co se vám na aplikaci líbilo? Co chcete do příště změnit nebo doplnit?'] != '']
    app_feedback_df[['Používali jste Animefest aplikaci?', 'Narazili jste při používání aplikace na závažné problémy?']] = app_feedback_df[['Používali jste Animefest aplikaci?', 'Narazili jste při používání aplikace na závažné problémy?']].fillna('-')
    app_feedback_html = app_feedback_df.to_html(index=False)

    app_feedback_2_df = df_form[['Používali jste Animefest aplikaci?', 'Narazili jste při používání aplikace na závažné problémy?', 'S čím jste měli problémy?', 'Jaké funkce chcete přidat nebo změnit?']]
    app_feedback_2_df = app_feedback_2_df[app_feedback_2_df[['S čím jste měli problémy?', 'Jaké funkce chcete přidat nebo změnit?']].notnull().any(axis=1)]
    app_feedback_2_df = app_feedback_2_df[(app_feedback_2_df[['S čím jste měli problémy?', 'Jaké funkce chcete přidat nebo změnit?']].fillna('') != '').any(axis=1)]
    app_feedback_2_df[['Používali jste Animefest aplikaci?', 'Narazili jste při používání aplikace na závažné problémy?']] = app_feedback_2_df[['Používali jste Animefest aplikaci?', 'Narazili jste při používání aplikace na závažné problémy?']].fillna('-')
    app_feedback_2_html = app_feedback_2_df.to_html(index=False)

    per_column_comments = {}
    for column in app_mention_columns:        
        comments_ser = df_form[has_app_mention(df_form[column])][column]
        #comments_ser = comments_ser.append(df_app[(df_app['název'] == column) & df_app['komentář'].str.lower().str.contains('moder')]['komentář'])
        # todo: spojit s komentáři z appky
        per_column_comments[column] = comments_ser.to_frame(name='komentáře').to_html(index=False)
        
    data_root = osp.join(os.getcwd(), 'app_feedback')
    pdf_path = data_root
    os.makedirs(pdf_path, exist_ok=True)
    'Co se vám na aplikaci líbilo? Co chcete do příště změnit nebo doplnit?'
    env = Environment(loader=FileSystemLoader(os.getcwd()), trim_blocks=True)
    template = env.get_template('app-template.jinja2')
    html_report_path = osp.join(pdf_path, 'app-report.html')
    template.stream(bar_chart_usage_path=osp.join(data_dir, 'app_usage.png'),
                    bar_chart_bugs_path=osp.join(data_dir, 'app_bugs.png'),
                    par_cat_path=img_path,
                    per_column_comments=per_column_comments,
                    app_feedback_html=app_feedback_html,
                    app_feedback_2_html=app_feedback_2_html,
                   ).dump(html_report_path)

    chrome_exe = '"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"'
    pdf_file = osp.join(pdf_path, 'app-report.pdf')
    command = f'{chrome_exe} --headless --disable-gpu --no-sandbox --print-to-pdf="{pdf_file}" "{html_report_path}"'
    os.popen(command)

generate_pdf()

In [ ]:
# todo: přidat ještě sloupce S čím jste měli problémy?	Jaké funkce chcete přidat nebo změnit?
app_feedback_2_df = df_form[['Používali jste Animefest aplikaci?', 'Narazili jste při používání aplikace na závažné problémy?', 'S čím jste měli problémy?', 'Jaké funkce chcete přidat nebo změnit?']]
app_feedback_2_df = app_feedback_2_df[app_feedback_2_df[['S čím jste měli problémy?', 'Jaké funkce chcete přidat nebo změnit?']].notnull().all(axis=1)]
app_feedback_2_df = app_feedback_2_df[(app_feedback_2_df[['S čím jste měli problémy?', 'Jaké funkce chcete přidat nebo změnit?']].fillna('') != '').all(axis=1)]


In [ ]:
app_feedback_2_df[['S čím jste měli problémy?', 'Jaké funkce chcete přidat nebo změnit?']].notnull().all(axis=1).head()

In [ ]:
app_feedback_2_df.head()